In [1]:
from keras.layers import SimpleRNN
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.layers import Dense
import numpy as np
import sys
sys.path.append("/data/keras/npl/")
from baidunpl import BaiduNpl
max_features=10000
maxlen=5
batch_size=32
(input_train,y_train),(input_test,y_test)=imdb.load_data(num_words=max_features)
print(y_train.shape)
#print(input_train)
input_train=sequence.pad_sequences(input_train,maxlen=maxlen)
input_test=sequence.pad_sequences(input_test,maxlen=maxlen)
print(input_train.shape)
print(input_train[0])


Using TensorFlow backend.


(25000,)
(25000, 5)
[  16 5345   19  178   32]


In [2]:
client=BaiduNpl()
model = Sequential()
model.add(LSTM(32))
# model.add(Dense(1, activation='sigmoid'))
model.add(Dense(64,activation='relu'))
model.add(Dense(5,activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

input_train=["后天天气如何","明天天气如何","下周一的天气",
             "我要去江门","我要去北京故宫的中心","导航到广州长隆",
             "我想听张韶涵的歌","来一首张学友的歌","我想听卓依婷的歌",
             "六点提醒我开会","五分钟后提醒睡觉","下午两点提醒我上学",
             "感冒了吃什么药","发烧了怎么办","眼睛不舒服怎么处理"]
y_train=np.array([[1,0,0,0,0],[1,0,0,0,0],[1,0,0,0,0],#问天气
                  [0,1,0,0,0],[0,1,0,0,0],[0,1,0,0,0],#导航
                  [0,0,1,0,0],[0,0,1,0,0],[0,0,1,0,0],#唱歌
                  [0,0,0,1,0],[0,0,0,1,0],[0,0,0,1,0],#提醒功能
                  [0,0,0,0,1],[0,0,0,0,1],[0,0,0,0,1]#医疗咨询
                 ])
input_train2=[]
for one_train in input_train:
    vecs=client.getVecFromJuzi(one_train)
    input_train2.append(vecs)
input_train2=np.array(input_train2)

input_train2=sequence.pad_sequences(input_train2,dtype="float",maxlen=20,padding='post',truncating='post',)
print(input_train2.shape)
history = model.fit(input_train2, y_train,epochs=15,batch_size=5)

(15, 20, 1024)
Epoch 1/15
15/15 [==============================] - 1s 85ms/step - loss: 1.6093 - acc: 0.2667
Epoch 2/15
15/15 [==============================] - 0s 6ms/step - loss: 1.5732 - acc: 0.6000
Epoch 3/15
15/15 [==============================] - 0s 6ms/step - loss: 1.4941 - acc: 0.7333
Epoch 4/15
15/15 [==============================] - 0s 7ms/step - loss: 1.3834 - acc: 0.8000
Epoch 5/15
15/15 [==============================] - 0s 7ms/step - loss: 1.2718 - acc: 0.8000
Epoch 6/15
15/15 [==============================] - 0s 7ms/step - loss: 1.1932 - acc: 0.8667
Epoch 7/15
15/15 [==============================] - 0s 7ms/step - loss: 1.0996 - acc: 0.9333
Epoch 8/15
15/15 [==============================] - 0s 7ms/step - loss: 1.0102 - acc: 0.9333
Epoch 9/15
15/15 [==============================] - 0s 7ms/step - loss: 0.9202 - acc: 1.0000
Epoch 10/15
15/15 [==============================] - 0s 7ms/step - loss: 0.8283 - acc: 1.0000
Epoch 11/15
15/15 [==============================] - 

In [6]:
tests=["星期二天气","导航到动物园","我想听周杰伦的歌","五点提醒我","得了结膜炎怎么处理"]
vecs2=[]
for test in tests: 
    vecs=client.getVecFromJuzi(test)  
    vecs2.append(vecs)
vecs=np.array(vecs2)
vecs2=sequence.pad_sequences(vecs2,dtype="float",maxlen=20,padding='post',truncating='post',)
print(vecs2.shape)
print(vecs2[0])
preds = model.predict(vecs2)
print(preds)

(5, 20, 1024)
[[-1.11579  -0.178418  1.74141  ... -0.318689 -1.43508   0.107404]
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 ...
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]]
[[0.6742539  0.09278884 0.04856174 0.10723553 0.07715999]
 [0.06700046 0.5853737  0.19827688 0.04617331 0.10317557]
 [0.00415179 0.04095656 0.92586887 0.00392235 0.0251005 ]
 [0.09371153 0.02938469 0.04143962 0.5146642  0.32079998]
 [0.01803083 0.02271373 0.1022286  0.2444688  0.612558  ]]


In [4]:
# import matplotlib.pyplot as plt
# acc=history.history['acc']
# val_acc=history.history['val_acc']
# loss=history.history['loss']
# val_loss=history.history['val_loss']
# epochs=range(1,len(acc)+1)
# plt.plot(epochs,acc,'bo',label='Training acc')
# plt.plot(epochs,val_acc,'b',label='Validation acc')
# plt.legend()
# plt.figure()
# plt.show()